- Policy Gradient를 구현해보자~
- 카트폴 이용 - OpenAI의 gym

In [2]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

### 하이퍼 파라미터 설정

In [4]:
learning_rate = 0.0002 #학습률 : 신경망 파라미터를 얼마나 업데이트할 것인가!
gamma = 0.98 # 할인률(Discount Factor)

### 정책 신경망 정의

In [5]:
env = gym.make("CartPole-v1")
print(env.observation_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [ ]:
class Policy(nn.Moule):
    def __init__(self):
        super(Policy, self).__init___()
        self.data =[]
        self.gamma = 0.99

        # 모델 생성 부분
        self.fc1 = nn.Linear(4, 128) # 4차원을 받아서 128차원으로 변경
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return 0
    
    def put_data(self, item):
        self.data.append(item)
    
    def train(self):
        R=0
        for r, log_prob in self.data[::-1]:
            R = r+R*


### 메인 실행 함수

In [ ]:
def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    avg_t =0 

    for n_epi in range(10000):
        obs = env.reset() # 환경의 첫 상태!
        for t in range(600): # 한 에피소드 안에서 600 iteration
            obs = torch.tensor(obs, dtype=torch.float) # 4차원 벡터!
            out = pi(obs)
            # sampling을 해야 함
            m = Categorical(out) # 확률 분포 모델
            action = m.sample() #그 확률에 비례해서 action을 뽑아줌. ex. tensor(0)
            obs, r, done, info = env.step(action.item())
            pi.put_data((r, torch.log(out[action]))) # policy 안에 데이터를 모아두는 것. 현재의 reward와 그것에 대한 내가 한 action = log\pi_\theta(s,ka)
            if done : # 다 끝나면 멈춰라!
                break
        avg_t += t # 여기서의 t는 몇 스텝을 버텼냐!
        pi.train() # 에피소드 끝났으니까 policy 학습시켜라
        if n_epi%20==0 and n_epi!=0:
            print("# of episode :{}, Avg timestep : {}".format(n_epi, avg_t/20.0))
            avg_t =0
    env.close()